In [13]:
import numpy as np              # Python's standard numerical library
import matplotlib.pyplot as plt # Python's scientific visualization library
from matplotlib.widgets import Cursor
import matplotlib.colors as mcolors
import matplotlib.animation as animation
from matplotlib.path import Path
from ipywidgets import interact # interactive plots in Jupyter Notebook
import pyidi                    # Python HSC data analysis library
# from scipy.interpolate import make_interp_spline, BSpline
import pickle as pk
import os
from pixel_setter2 import PixelSetter#, play_video, detect_peaks
from skimage import feature, measure
# from skimage.morphology import skeletonize
from skimage.feature import match_template
from mpl_toolkits import mplot3d
import dill
import cv2 
from scipy.signal import correlate2d
%matplotlib qt

Open the file to analyze and set an analysis method (Lucas-Kanade implementation)

In [14]:
folder_path = 'G:\\.shortcut-targets-by-id\\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\\Illimited Lab Projects\\Spiders\\Other\\Blackledge_media'
file_name = "Ar.tri.14_11.15.22_30mm_natfq(full)"
file_name_video = file_name + ".mp4"
file_path = os.path.join(folder_path, file_name_video)
cap = cv2.VideoCapture(file_path)

ret, frame = cap.read()
all_frames = []
while ret:
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    all_frames.append(gray_frame[np.newaxis, :, :])
    ret, frame = cap.read()

all_frames = np.concatenate(all_frames, axis=0) 
cap.release()

video = pyidi.pyIDI(all_frames) #(n time points, image height, image width)
video.N = np.shape(all_frames)[0]
video.info['Total Frame'] = video.N
video.info['Frame Rate'] = 1000
video.info['Image Width'] = np.shape(all_frames)[2]
video.info['Image Height'] = np.shape(all_frames)[1]
video.image_width = video.info['Image Width']
video.image_height = video.info['Image Height']
mraw = video.mraw
video.info
del all_frames

Plot a frame in the video:

In [15]:
%matplotlib qt
sequential_image_n = 0
still_image = mraw[sequential_image_n]

sigma_vec = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])*3
fig, ax = plt.subplots(ncols=3, nrows=3, figsize=(10, 5))
for ax_i, sigma in enumerate(sigma_vec):
    row = ax_i // 3
    col = ax_i % 3
    edges = feature.canny(still_image, sigma=sigma)
    ax[row, col].imshow((2**16-1)-edges, cmap='gray')
    # skeleton = skeletonize(edges)
    # contours = measure.find_contours(edges, sigma)
    # ax[row, col].imshow(still_image, cmap='gray')
    # for contour in contours:
        # ax[row, col].plot(contour[:, 1], contour[:, 0], linewidth=2)


In [ ]:
pix_set = PixelSetter(still_image, file_name = file_name)
# pix_set.get_tracking_pixels()
pix_set.choose_reference_centers()


In [ ]:
pix_set.combine_tracking_points()

In [ ]:
pix_set.save(file_name)

In [ ]:
tp = np.array([[ 686,  952],
       [ 349,  958],
       [ 785,  842],
       [ 452, 1041],
       [ 373,  906],
       [ 596,  942],
       [ 519,  987],
       [ 758,  842],
       [ 638,  935],
       [ 595,  964],
       [ 731,  852],
       [ 396, 1029],
       [ 662,  892],
       [ 505,  945],
       [ 372,  910],
       [ 848, 1037],
       [ 591,  868],
       [ 393,  960],
       [ 430,  869],
       [ 589, 1042],
       [ 861,  957],
       [ 849, 1001],
       [ 822,  984],
       [ 652,  838],
       [ 397,  966],
       [ 408,  877],
       [ 575,  963],
       [ 548, 1018],
       [ 337, 1073],
       [ 512,  943],
       [ 387,  921],
       [ 658,  997],
       [ 827,  964],
       [ 497,  918],
       [ 672, 1017],
       [ 636, 1005],
       [ 432,  946],
       [ 716,  961],
       [ 459,  936],
       [ 525,  881],
       [ 862, 1042],
       [ 600, 1042],
       [ 511,  964],
       [ 701,  954],
       [ 398,  925],
       [ 790, 1022],
       [ 535, 1025],
       [ 486,  912],
       [ 755, 1046],
       [ 487,  975],
       [ 434,  996],
       [ 551, 1014],
       [ 350,  844],
       [ 563,  988],
       [ 625,  866],
       [ 387,  920],
       [ 371,  912],
       [ 780,  842],
       [ 856,  999],
       [ 798,  985],
       [ 462,  950],
       [ 394, 1035],
       [ 752, 1036],
       [ 304,  998],
       [ 435,  914],
       [ 638, 1041],
       [ 643,  976],
       [ 472,  847],
       [ 508,  931],
       [ 392,  976],
       [ 321,  934],
       [ 606,  945],
       [ 386,  923],
       [ 375,  923],
       [ 681,  877],
       [ 314,  925],
       [ 521,  986],
       [ 398,  906],
       [ 645,  879],
       [ 463,  910],
       [ 736,  919],
       [ 627,  851],
       [ 647,  838],
       [ 488,  930],
       [ 500,  904],
       [ 374,  926],
       [ 421,  867],
       [ 718,  992],
       [ 583,  947],
       [ 386,  927],
       [ 361, 1010],
       [ 410,  884],
       [ 831,  905],
       [ 349,  883],
       [ 494,  972],
       [ 584,  875],
       [ 617,  972],
       [ 602,  920],
       [ 333,  884],
       [ 704,  949],
       [ 488,  943],
       [ 363,  930],
       [ 359,  896],
       [ 350,  847],
       [ 389,  915],
       [ 551,  846],
       [ 464,  945],
       [ 486,  856],
       [ 629, 1012],
       [ 435,  890],
       [ 687,  864],
       [ 507,  906],
       [ 333,  879],
       [ 307, 1006],
       [ 648,  872],
       [ 374,  925],
       [ 392,  979],
       [ 592, 1004],
       [ 490,  947],
       [ 591, 1026],
       [ 419,  941],
       [ 554, 1004],
       [ 805,  842],
       [ 456,  837],
       [ 313, 1072],
       [ 536,  973],
       [ 323, 1076],
       [ 344,  879],
       [ 801,  984],
       [ 427,  895],
       [ 640, 1048],
       [ 317,  907],
       [ 565,  981],
       [ 366,  921],
       [ 464,  953],
       [ 478,  851],
       [ 508,  870],
       [ 742, 1046],
       [ 374,  897],
       [ 632, 1021],
       [ 373,  928],
       [ 344,  892],
       [ 644, 1004],
       [ 474,  840],
       [ 625, 1000],
       [ 696, 1051],
       [ 365,  924],
       [ 590, 1042],
       [ 388,  944],
       [ 648,  983],
       [ 431,  896],
       [ 484,  917],
       [ 804,  998],
       [ 424,  875],
       [ 341,  859],
       [ 496,  891],
       [ 433,  951],
       [ 540,  915],
       [ 376,  920],
       [ 373,  905],
       [ 460,  919],
       [ 853,  993],
       [ 436,  970],
       [ 561,  992],
       [ 418,  931],
       [ 798,  932],
       [ 595,  972],
       [ 333,  841],
       [ 350,  912],
       [ 778,  864],
       [ 302,  993],
       [ 457,  885],
       [ 729,  953],
       [ 855,  940],
       [ 456,  925],
       [ 336,  869],
       [ 448,  930],
       [ 565,  948],
       [ 400,  890],
       [ 464,  902],
       [ 314,  948],
       [ 429,  935],
       [ 452,  909],
       [ 832, 1041],
       [ 473,  959],
       [ 443,  964],
       [ 446,  898],
       [ 458,  872],
       [ 632, 1006],
       [ 491,  896],
       [ 418,  912],
       [ 345,  940],
       [ 437,  925],
       [ 406,  928],
       [ 620,  897],
       [ 452,  913],
       [ 514,  992],
       [ 497,  930],
       [ 784, 1023],
       [ 473,  981],
       [ 331,  888],
       [ 447,  947],
       [ 353,  903],
       [ 521,  905],
       [ 561,  941],
       [ 634, 1029],
       [ 853,  978],
       [ 386,  936],
       [ 468, 1024],
       [ 418,  916],
       [ 418,  934],
       [ 666, 1008],
       [ 594,  992],
       [ 501,  959],
       [ 441,  908],
       [ 463,  914],
       [ 399,  920],
       [ 335,  874],
       [ 503,  968],
       [ 477,  839],
       [ 798, 1020],
       [ 455,  946],
       [ 624,  876],
       [ 442,  845],
       [ 364,  925],
       [ 428,  928],
       [ 434, 1008],
       [ 637,  931],
       [ 629,  936],
       [ 538,  836],
       [ 357,  913],
       [ 470,  930],
       [ 387,  923],
       [ 493,  941],
       [ 386,  967],
       [ 398,  932],
       [ 594,  996],
       [ 443,  931],
       [ 387,  941],
       [ 420,  947],
       [ 836, 1032],
       [ 321,  852],
       [ 636,  845],
       [ 428,  923],
       [ 471,  903],
       [ 718, 1009],
       [ 519,  961],
       [ 543,  928],
       [ 694, 1051],
       [ 350,  851],
       [ 321,  964],
       [ 394,  948],
       [ 764, 1003],
       [ 636,  966],
       [ 318,  913],
       [ 407, 1012],
       [ 431,  942],
       [ 401,  938],
       [ 420,  951],
       [ 561,  899],
       [ 306,  965],
       [ 363,  929],
       [ 863,  977],
       [ 745, 1021],
       [ 595,  933],
       [ 301,  990],
       [ 593, 1000],
       [ 668, 1012],
       [ 662,  964],
       [ 488,  937],
       [ 893, 1004],
       [ 499,  946],
       [ 488,  955],
       [ 386,  925],
       [ 518,  928],
       [ 528,  883],
       [ 455,  908],
       [ 869, 1042],
       [ 476,  958],
       [ 829,  987],
       [ 630, 1016],
       [ 496,  936],
       [ 334,  874],
       [ 653,  990],
       [ 579,  952],
       [ 780,  951],
       [ 480,  937],
       [ 385,  928],
       [ 386,  893],
       [ 669,  845],
       [ 475,  849],
       [ 477,  931],
       [ 622,  886],
       [ 439,  941],
       [ 428, 1002],
       [ 581,  863],
       [ 613,  913],
       [ 613,  922],
       [ 496,  913],
       [ 622,  989],
       [ 314,  863],
       [ 334,  878],
       [ 428,  903],
       [ 438,  935],
       [ 442,  914],
       [ 729, 1007],
       [ 616,  968],
       [ 388,  970],
       [ 333,  863],
       [ 704,  946],
       [ 614,  909],
       [ 365,  851],
       [ 882,  970],
       [ 849, 1031],
       [ 790,  963],
       [ 633, 1025],
       [ 408,  916],
       [ 408,  934],
       [ 721,  944],
       [ 773, 1032],
       [ 778,  872],
       [ 545, 1024],
       [ 383,  962],
       [ 355,  909],
       [ 371, 1015],
       [ 469,  894],
       [ 881,  866],
       [ 395,  963],
       [ 596,  953],
       [ 430,  939],
       [ 460,  940],
       [ 812,  842],
       [ 640,  888],
       [ 410,  938],
       [ 625,  872],
       [ 350,  951],
       [ 581,  951],
       [ 469,  889],
       [ 877,  982],
       [ 596,  948],
       [ 636,  895],
       [ 388,  919],
       [ 590, 1026],
       [ 329,  842],
       [ 897, 1043],
       [ 406,  945],
       [ 608,  949],
       [ 452,  930],
       [ 381,  888],
       [ 390,  973],
       [ 819, 1016],
       [ 636, 1034],
       [ 399,  969],
       [ 438,  879],
       [ 349,  959],
       [ 402,  974],
       [ 332,  888],
       [ 826,  996],
       [ 436,  885],
       [ 464,  909],
       [ 589,  940],
       [ 495,  914],
       [ 468,  897],
       [ 764,  895],
       [ 804,  842],
       [ 621,  986],
       [ 532,  908],
       [ 467,  937],
       [ 754, 1030],
       [ 481,  853],
       [ 895,  987],
       [ 407,  922],
       [ 543,  950],
       [ 784, 1003],
       [ 390,  954],
       [ 876,  866],
       [ 539,  952],
       [ 332,  860]])

In [ ]:
# tp = np.array(list(pix_set.tracking_points))
center = np.array(still_image.shape[:2])/2
# radius = 0.75*np.linalg.norm(center)
# distances = np.linalg.norm(tp - center, axis=1)
# inside_web = (distances < radius)
hor_section = (tp[:, 1] > 835) & (tp[:, 1] < 1100)
ver_section = (tp[:, 0] > 300) & (tp[:, 0] < 900)
tp_lim = tp[hor_section & ver_section]

fig, ax = plt.subplots(figsize=(10, 10))

# plot tp
plt.scatter(center[1], center[0])
plt.scatter(tp[:, 1], tp[:, 0], c='red', s=10, label='tp')

# plot tp_lim with markers on top of tp
plt.scatter(tp_lim[:, 1], tp_lim[:, 0], c='green', s=5, label='tp_lim')
ax.set_xlim([0, still_image.shape[1]])
ax.set_ylim([still_image.shape[0] , 0])
ax.axis('equal')
# show the plot
plt.show()


In [ ]:
tp_lim = np.delete(tp_lim, 122, axis=0)
tp_lim = np.delete(tp_lim, 363, axis=0)
tp_lim = np.delete(tp_lim, 168, axis=0)
tp_lim = np.delete(tp_lim, 287, axis=0)
tp_lim = np.delete(tp_lim, 7, axis=0)
tp_lim = np.delete(tp_lim, 22, axis=0)
tp_lim = np.delete(tp_lim, 326, axis=0)
tp_lim = np.delete(tp_lim, 80, axis=0)
tp_lim = np.delete(tp_lim, 218, axis=0)
tp_lim = np.delete(tp_lim, 55, axis=0)
tp_lim = np.delete(tp_lim, 365, axis=0)
tp_lim = np.delete(tp_lim, 235, axis=0)
tp_lim = np.delete(tp_lim, 317, axis=0)
tp_lim = np.delete(tp_lim, 2, axis=0)
tp_lim = np.delete(tp_lim, 8, axis=0)
tp_lim = np.delete(tp_lim, 99, axis=0)
tp_lim = np.delete(tp_lim, 308, axis=0)
tp_lim = np.delete(tp_lim, 329, axis=0)
tp_lim = np.delete(tp_lim, 53, axis=0)
tp_lim = np.delete(tp_lim, 209, axis=0)
tp_lim = np.delete(tp_lim, 53, axis=0)
tp_lim = np.delete(tp_lim, 162, axis=0)
video.set_points(tp_lim)
video.set_method('lk')
video.show_points()


In [ ]:
d = video.get_displacements(autosave=False)
# video.method.valid_points
d = video.displacements
d


In [ ]:
import pickle
root = 'C:/Users/thijsmas/Documents/GitHub/pyidi_data/'
path = os.path.join(root, file_name+'_d'+'.pkl')
with open(path, 'wb') as f:
    pickle.dump({'displacement':d, 'tracking points': tp_lim}, f)


In [ ]:
from pixel_setter import play_video
# tp = np.array(list(pix_set.tracking_points))
td = d +  tp_lim.reshape(len(tp_lim),1,2)
play_video(video, range(1,len(pix_set.tracking_points)), interval=30, points=td)

Mask points with high amplitude, outside of web, and irregular vibration

In [ ]:
# center = np.array(still_image.shape[:2])/2
# radius = 0.75*np.linalg.norm(center)
# distances = np.linalg.norm(tp - center, axis=1)
# inside_web = (distances < radius) #& tp[0,:] > still_image.shape[0]*.1

low_amplitude = np.max(np.linalg.norm(d, axis=2),1)<50
play_video(video, range(500,len(pix_set.tracking_points)), interval=30, points=td[low_amplitude,:,:])

In [ ]:
sigma_vec = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])*3
fig, ax = plt.subplots(ncols=3, nrows=3, figsize=(10, 5))
for ax_i, sigma in enumerate(sigma_vec):
    row = ax_i // 3
    col = ax_i % 3
    contours = measure.find_contours(still_image, sigma)
    ax[row, col].imshow(still_image, cmap='gray')
    for contour in contours:
        ax[row, col].plot(contour[:, 1], contour[:, 0], linewidth=2)

Find the gradient off the intensity accross the still figure

In [ ]:
gy, gx = np.gradient(still_image, edge_order=2)
g_norm = np.sqrt(gx**2 + gy**2)
arg_g  = g_norm > 15000
pix_gi, pix_gj = np.where(arg_g)
fig, ax = plt.subplots()
ax.imshow(still_image, cmap='gray')
ax.plot(pix_gj, pix_gi, 'r.')

In [ ]:
pix_set.generate_reference_image(plot=True, seperate_save=True)

In [ ]:
sliders = pix_set.cross_correlate_all()

In [ ]:
# white_value, black_value = pix_set.get_white_black_value()
# pix_set.plot_histogram(bins=50)

# fig, ax = plt.subplots()

# colors = [(1, 0, 0), (0, 1, 0)]
# markers = ['o', 's', 'v', '^', 'D', 'p', '*', 'X', 'd', 'h']
# markersize = 10
# cmap = mcolors.LinearSegmentedColormap.from_list('red_to_green', colors, N=256)

# still_image_temp = still_image.copy() - still_image.mean()
# Low_I = still_image < 20000
# # reference_crosses.append(still_image_temp[263:270, 334:341].astype(np.float64))
# corr_max = np.zeros((still_image.shape[0], still_image.shape[1]))
# for i, reference_cross in enumerate(pix_set.ref_imgs):
#     # template = reference_crosses[0].astype(np.float64)
#     # template -= template.mean()
#     # corr = correlate2d(still_image_temp, template, boundary='symm', mode='same')
#     # corr = match_template(still_image, reference_cross, pad_input=True)
#     # high_corr = corr > 0.75
#     # corr = (corr-np.min(corr)) / np.max(corr-np.min(corr))
#     # peaks_local = detect_peaks(corr)
#     # y, x = np.where(peaks_local & high_corr & Low_I) # & high_corr
#     x, y, corr = pix_set.cross_correlate(reference_cross)
#     ax.scatter(x, y, marker=markers[i % len(markers)], s=markersize*3)
#     markersize -= 1
#     corr_max = np.maximum(corr_max, corr)



ax.imshow(still_image, cmap='gray')
num_plots = len(pix_set.ref_imgs)
if num_plots == 1:
    fig, ax = plt.subplots()
    ax.imshow(pix_set.ref_imgs[0], cmap='gray')
elif num_plots == 2:
    fig, ax = plt.subplots(ncols=2)
    for i, ref_cross in enumerate(pix_set.ref_imgs):
        ax[i].imshow(ref_cross, cmap='gray')
else:
    num_cols = np.ceil(np.sqrt(num_plots))
    num_rows, remainder = divmod(num_plots, num_cols)
    if remainder > 0:
        num_rows += 1

    fig, ax = plt.subplots(nrows = int(num_rows), ncols= int(num_cols))
    for i, ref_cross in enumerate(pix_set.ref_imgs):
        row = int(i // num_cols)
        col = int(i % num_cols)
        ax[row, col].imshow(ref_cross, cmap='gray')
plt.show()


In [ ]:
template = still_image[263:270, 334:341]
result = match_template(still_image, template)
ij = np.unravel_index(np.argmax(result), result.shape)
x, y = ij[::-1]
fig, ax = plt.subplots()
ax.imshow(still_image, cmap='gray')
hcoin, wcoin = template.shape
rect = plt.Rectangle((x, y), wcoin, hcoin, edgecolor='r', facecolor='none')
ax.add_patch(rect)
plt.show()

Set limits on the Pixel intensity, Gradient x, and Gradient y to define a Region of Interest (ROI)
By drawing polygons on the figure and pressing enter, parts of the ROI can be removed.

In [ ]:
%matplotlib qt
arg_pi = still_image < 70000
arg_gy = np.linalg.norm([gy, gx], axis=0) > 28000
args = arg_pi & arg_gy #| arg_gx
pix_i, pix_j = np.where(args)


pixelsetter = PixelSetter(still_image, pix_i, pix_j, file_name, sequential_image_n)
pixelsetter.set_method('by_hand')
pixelsetter.get_tracking_pixels()


In [ ]:
pixelsetter.plot_histogram(inside_polygon=True)
pixelsetter.save()

In [ ]:
# del pixelsetter
pixelsetter = PixelSetter.load(file_name)
sequential_image_n = pixelsetter.frame_nr
pix_i = pixelsetter.pix_i
pix_j = pixelsetter.pix_j

Since the above process can be time consuming, the code below can be used to replot the ROI

In [ ]:
# pix_i, pix_j, np.linalg.norm([gy, gx], axis=0).max()

In [ ]:
%matplotlib qt
# mraw_range = (sequential_image_n, sequential_image_n+800) 
# n_frames = mraw_range[1] - mraw_range[0]

# fig, ax = plt.subplots(figsize=(15, 5))
# ax.imshow(still_image, cmap='gray')
# plt.plot(pix_j, pix_i, 'r.', markersize=1)
video.set_points(np.stack((pix_i, pix_j), axis=-1))
video.set_method('lk')
video.method.configure()  #mraw_range=(sequential_image_n, video.N) reference_range=(1,2)
# print((video.method.start_time, video.method.stop_time))
video.show_points()


In [ ]:
# %matplotlib inline
# ref_img = video.method.reference(video.mraw[video.method.reference_range[0]: video.method.reference_range[1]], video.method.subset_size)
# fig, ax = plt.subplots()
# ax.imshow(still_image, cmap='gray')
# plt.show()

In [ ]:
d = video.get_displacements()
# video.method.valid_points
d = video.displacements
d

In [ ]:
%matplotlib qt
fig, ax = plt.subplots()
for point in np.arange(d.shape[0]): #[video.method.valid_points]
    ax.plot(np.arange(video.N), d[point, :, 0], label='point {} - dir 0'.format(point)) #video.info['Record Rate(fps)']*
    ax.plot(np.arange(video.N), d[point, :, 1], label='point {} - dir 1'.format(point)) #video.info['Record Rate(fps)']*
# ax.legend()
# ax.set_ylim(-50, 50)
ax.set_xlabel('frame number')
ax.set_ylabel('displacement [pixels]')
# ax.set_xlim(0, n_frames)

In [ ]:
# Add pix_i and pix_j to displacement to get the tracked position
# mask = np.any(d > 3, axis=(1,2))
tp = (np.stack((pix_i, pix_j), axis=-1)).reshape(len(pix_i),1,2) + d # d[:,:n_frames,:]
# tp[mask, :, :] = np.nan
ani = play_video(video, range(1,799), interval=60, points=tp ) #axis=([720,760],[300,230])axis = ([450,350],[460,560])
# ani = play_video(video, range(10000), interval=30)
# writer = animation.FFMpegWriter(fps=60)
# path = 'C:/Users/thijs/Documents/GitHub/pyidi_data/anim/'
# ani.save(path+file_name + '_LK.mp4', writer=writer)

In [ ]:
import os

folder_path = 'H:/My Drive/PHD/HSC'
all_files_drive = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_drive.append(file)

folder_path = 'D:/HSC'
all_files_T7 = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_T7.append(file)

folder_path = 'F:/'
all_files_Sandisk = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_Sandisk.append(file)


folder_path = 'E:/thijs'
all_files_Passport = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_Passport.append(file)

folder_path = 'C:/Users/thijs/Documents/HSC'
all_files_internal = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        all_files_internal.append(file)


In [ ]:
unique_drive_files = set(all_files_drive) - set(all_files_T7) - set(all_files_Sandisk) - set(all_files_Passport) - set(all_files_internal)
unique_drive_files